In [1]:
import pymc as pm
import matplotlib.pyplot as plt
import arviz as az
import pandas as pd
from scipy import special, stats
import numpy as np
import seaborn as sns 

WARNING (pytensor.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (pytensor.configdefaults): g++ not detected!  PyTensor will be unable to compile C-implementations and will default to Python. Performance may be severely degraded. To remove this warning, set PyTensor flags cxx to an empty string.
WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
a = pd.read_csv("C:/Users/jacqu/Downloads/algae.csv")

In [3]:
a.mxPH = a.mxPH.replace(np.NAN, a.mxPH.mean())
a.mnO2 = a.mnO2.replace(np.NAN, a.mnO2.mean())
a.Cl = a.Cl.replace(np.NAN, a.Cl.mean())
a.NO3 = a.NO3.replace(np.NAN, a.NO3.mean())
a.NH4 = a.NH4.replace(np.NAN, a.NH4.mean())
a.oPO4 = a.oPO4.replace(np.NAN, a.oPO4.mean())
a.PO4 = a.PO4.replace(np.NAN, a.PO4.mean())
a.Chla = a.Chla.replace(np.NAN, a.Chla.mean())

a.mxPH = (a.mxPH-a.mxPH.mean())/a.mxPH.std()
a.mnO2 = (a.mnO2-a.mnO2.mean())/a.mnO2.std()
a.Cl = (a.Cl-a.Cl.mean())/a.Cl.std()
a.NO3 = (a.NO3-a.NO3.mean())/a.NO3.std()
a.NH4 = (a.NH4-a.NH4.mean())/a.NH4.std()
a.oPO4 = (a.oPO4-a.oPO4.mean())/a.oPO4.std()
a.PO4 = (a.PO4-a.PO4.mean())/a.PO4.std()
a.Chla = (a.Chla-a.Chla.mean())/a.Chla.std()

a[['spring', 'summer', 'winter']] = pd.get_dummies(a.season, dtype=int, drop_first=True)
a[['medium','small']] = pd.get_dummies(a['size'], dtype=int, drop_first=True)
a[['l' ,'m']] = pd.get_dummies(a.speed, dtype=int, drop_first=True)

a.a1 = np.log(a.a1+1)

In [4]:
random_seed = 135573

spring_obs = a.loc[:, "spring"]
summer_obs = a.loc[:, "summer"]
winter_obs = a.loc[:, "winter"]
medium_obs = a.loc[:, "medium"]
small_obs = a.loc[:, "mxPH"]
l_obs = a.loc[:, "l"]
m_obs = a.loc[:, "m"]
mxPH_obs = a.loc[:, "mxPH"]
mnO2_obs = a.loc[:, "mnO2"]
Cl_obs = a.loc[:, "Cl"]
NO3_obs = a.loc[:, "NO3"]
NH4_obs = a.loc[:, "NH4"]
oPO4_obs = a.loc[:, "oPO4"]
PO4_obs = a.loc[:, "PO4"]
Chla_obs = a.loc[:, "Chla"]
a1_obs = a.loc[:, "a1"]

with pm.Model() as a_model:
    
    spring = pm.Data("spring", spring_obs)
    summer = pm.Data("summer", summer_obs)
    winter = pm.Data("winter", winter_obs)
    medium = pm.Data("medium", medium_obs)
    small = pm.Data("small", small_obs)
    l = pm.Data("l", l_obs)
    m = pm.Data("m", m_obs)
    mxPH = pm.Data("mxPH", mxPH_obs)
    mnO2 = pm.Data("mnO2", mnO2_obs)
    Cl = pm.Data("Cl", Cl_obs)
    NO3 = pm.Data("NO3", NO3_obs)
    NH4 = pm.Data("NH4", NH4_obs)
    oPO4 = pm.Data("oPO4", oPO4_obs)
    PO4 = pm.Data("PO4", PO4_obs)
    Chla = pm.Data("Chla", Chla_obs)

    nu = pm.Exponential("nu", 1/20)
    sigma = pm.Uniform("sigma", 0, 10)
    beta_0 = pm.Normal("beta_0", 0, 10)
    beta_spring = pm.Normal("beta_spring", 0, 10)
    beta_summer = pm.Normal("beta_summer", 0, 10)
    beta_winter = pm.Normal("beta_winter", 0, 10)
    beta_medium = pm.Normal("beta_medium", 0, 10)
    beta_small = pm.Normal("beta_small", 0, 10)
    beta_l = pm.Normal("beta_l", 0, 10)
    beta_m = pm.Normal("beta_m", 0, 10)
    beta_mxPH = pm.Normal("beta_mxPH", 0, 10)
    beta_mnO2 = pm.Normal("beta_mnO2", 0, 10)
    beta_Cl = pm.Normal("beta_Cl", 0, 10)
    beta_NO3 = pm.Normal("beta_NO3", 0, 10)
    beta_NH4 = pm.Normal("beta_NH4", 0, 10)
    beta_oPO4 = pm.Normal("beta_oPO4", 0, 10)
    beta_PO4 = pm.Normal("beta_PO4", 0, 10)
    beta_Chla = pm.Normal("beta_Chla", 0, 10)
    
    mu = pm.Deterministic("mu", beta_0 + \
                         beta_spring * spring + beta_summer * summer + \
                         beta_winter * winter + \
                         beta_medium * medium + beta_small * small + \
                         beta_l * l + beta_m * m + \
                         beta_mxPH * mxPH + beta_mnO2 * mnO2 + \
                         beta_Cl * Cl + beta_NO3 * NO3 + beta_NH4 * NH4 + \
                         beta_oPO4 * oPO4 + beta_PO4 * PO4 + beta_Chla * Chla)

    a1 = pm.StudentT("a1", mu=mu, sigma=sigma, nu=nu, observed = a1_obs)

    a_trace = pm.sample(target_accept=0.9, random_seed=random_seed, return_inferencedata=False)

C:\Users\jacqu\AppData\Local\Programs\Python\Python311\Lib\site-packages\pymc\data.py:433: UserWarning: The `mutable` kwarg was not specified. Before v4.1.0 it defaulted to `pm.Data(mutable=True)`, which is equivalent to using `pm.MutableData()`. In v4.1.0 the default changed to `pm.Data(mutable=False)`, equivalent to `pm.ConstantData`. Use `pm.ConstantData`/`pm.MutableData` or pass `pm.Data(..., mutable=False/True)` to avoid this warning.
  warnings.warn(
C:\Users\jacqu\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytensor\tensor\rewriting\elemwise.py:1028: UserWarning: Loop fusion failed because the resulting node would exceed the kernel argument limit.
  warn(
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
C:\Users\jacqu\AppData\Local\Programs\Python\Python311\Lib\site-packages\pytensor\tensor\rewriting\elemwise.py:1028: UserWarning: Loop fusion failed because the resulting node would exceed the kernel argument limit.
  warn(
C:\Users\jacqu\

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 47767 seconds.
